In [1]:
import pandas as pd
import os
import chardet

imports_folder = r'C:\Users\isaac\Downloads\USMCA_Industry_Mexico_Imports'
exports_folder = r'C:\Users\isaac\Downloads\USMCA_Industry_Mexico_Exports'

def load_and_process_data(imports_folder, exports_folder):
    all_data = []
    
    # Function to detect encoding
    def detect_encoding(file_path):
        with open(file_path, "rb") as f:
            result = chardet.detect(f.read(100000))  # Read first 100KB
            return result["encoding"]
    
    # Process imports
    for file in os.listdir(imports_folder):
        if file.endswith("_Mexico_Imports.csv"):
            state_name = file.replace("_Mexico_Imports.csv", "")
            file_path = os.path.join(imports_folder, file)  # Correct path joining
            
            encoding_detected = detect_encoding(file_path)  # Detect encoding
            
            df = pd.read_csv(file_path, encoding=encoding_detected, sep="\t", engine="python")
            df["State"] = state_name
            df["Flow"] = "Imports"
            all_data.append(df)
    
    # Process exports
    for file in os.listdir(exports_folder):
        if file.endswith("_Mexico_Exports.csv"):
            state_name = file.replace("_Mexico_Exports.csv", "")
            file_path = os.path.join(exports_folder, file)  # Correct path joining
            
            encoding_detected = detect_encoding(file_path)  # Detect encoding
            
            df = pd.read_csv(file_path, encoding=encoding_detected, sep="\t", engine="python")
            df["State"] = state_name
            df["Flow"] = "Exports"
            all_data.append(df)
    
    # Combine all data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)

    # Ensure proper sorting by product, state, and year
    combined_df = combined_df.sort_values(by=["Product Classification", "State"])

    # Convert year columns to numeric
    year_columns = [col for col in combined_df.columns if col.isdigit()]
    combined_df[year_columns] = (
        combined_df[year_columns]
        .replace("[\$,]", "", regex=True)  # Remove $ and commas
        .astype(float)  # Convert to float
    )
    
    # Interpolate missing values
    combined_df[year_columns] = combined_df[year_columns].interpolate(method="linear", axis=1)
    
    # Fill null values with 0
    combined_df = combined_df.fillna(0)
    
    return combined_df

# Call the function and print column names
industry_trade_df = load_and_process_data(imports_folder, exports_folder)
print(industry_trade_df.columns)

# Save to a new CSV file
industry_trade_df.to_csv("USMCA_Industry_Mexico_Trade.csv", index=False)


Index(['Product ', 'Flow', 'Country', 'Product Classification', 'State',
       'Unit', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
      dtype='object')


In [2]:
file_path = r'C:\Users\isaac\Downloads\USMCA_Industry_Mexico_Trade.csv'

# Reset index and ensure consistent column alignment
industry_trade_df = industry_trade_df.reset_index(drop=True)

industry_trade_df = industry_trade_df

# industry_trade_df.drop(columns=['Country', 'Product Classification', 'Unit'], inplace=True)

if os.path.exists(file_path):
    print("File Path Exists")
else:
    industry_trade_df.to_csv(file_path, index=False)
    print("CSV created successfully with industry imports and exports!")

File Path Exists


In [3]:
industry_trade_df.head(10)

,Product,Flow,Country,Product Classification,State,Unit,2009,2010,2011,2012,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,0--All Merchandise,Imports,Mexico,NAICS - 3,Alabama,Millions of USD,671.0,846.0,974.0,1186.0,...,2878.0,3169.0,3567.0,4133.0,4953.0,4153.0,5530.0,7901.0,8146.0,6897.0
1,336--Transportation Equipment,Imports,Mexico,NAICS - 3,Alabama,Millions of USD,280.0,439.0,527.0,626.0,...,1216.0,1562.0,1595.0,1440.0,1922.0,2016.0,2444.0,2952.0,3321.0,3442.0
2,211--Oil & Gas,Imports,Mexico,NAICS - 3,Alabama,Millions of USD,0.0,0.0,0.0,22.0,...,899.0,762.0,1088.0,1581.0,1559.0,855.0,1480.0,2909.0,2840.0,1414.0
3,331--Primary Metal Manufactures,Imports,Mexico,NAICS - 3,Alabama,Millions of USD,6.0,28.0,32.0,32.0,...,61.0,97.0,88.0,281.0,566.0,311.0,412.0,310.0,246.0,327.0
4,332--Fabricated Metal Products,Imports,Mexico,NAICS - 3,Alabama,Millions of USD,20.0,19.0,27.0,40.0,...,33.0,32.0,57.0,72.0,73.0,96.0,149.0,202.0,265.0,289.0
5,"333--Machinery, Except Electrical",Imports,Mexico,NAICS - 3,Alabama,Millions of USD,18.0,27.0,68.0,89.0,...,165.0,176.0,147.0,152.0,180.0,161.0,202.0,247.0,317.0,281.0
6,"335--Electrical Equipment, Appliances & Compon...",Imports,Mexico,NAICS - 3,Alabama,Millions of USD,54.0,78.0,73.0,78.0,...,127.0,106.0,109.0,142.0,146.0,156.0,189.0,321.0,283.0,259.0
7,325--Chemicals,Imports,Mexico,NAICS - 3,Alabama,Millions of USD,41.0,46.0,55.0,58.0,...,54.0,43.0,125.0,117.0,121.0,147.0,176.0,223.0,177.0,165.0
8,980--Goods Returned (Exports and Imports) and ...,Imports,Mexico,NAICS - 3,Alabama,Millions of USD,21.0,18.0,11.0,11.0,...,31.0,65.0,48.0,64.0,83.0,63.0,37.0,211.0,152.0,162.0
9,334--Computer & Electronic Products,Imports,Mexico,NAICS - 3,Alabama,Millions of USD,30.0,36.0,14.0,53.0,...,27.0,50.0,71.0,46.0,36.0,43.0,42.0,41.0,63.0,144.0


In [4]:
print(industry_trade_df.columns)

Index(['Product ', 'Flow', 'Country', 'Product Classification', 'State',
       'Unit', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
      dtype='object')


In [5]:
# Strip the whitespace from the 'Product ' column (with space)
industry_trade_df.columns = industry_trade_df.columns.str.strip()

# Now strip the whitespace from the 'Product' column
industry_trade_df['Product'] = industry_trade_df['Product'].str.strip()

# Check the first few rows to verify
print(industry_trade_df.head(60))


                                              Product     Flow Country  \
0                                  0--All Merchandise  Imports  Mexico   
1                       336--Transportation Equipment  Imports  Mexico   
2                                      211--Oil & Gas  Imports  Mexico   
3                     331--Primary Metal Manufactures  Imports  Mexico   
4                      332--Fabricated Metal Products  Imports  Mexico   
5                   333--Machinery, Except Electrical  Imports  Mexico   
6   335--Electrical Equipment, Appliances & Compon...  Imports  Mexico   
7                                      325--Chemicals  Imports  Mexico   
8   980--Goods Returned (Exports and Imports) and ...  Imports  Mexico   
9                 334--Computer & Electronic Products  Imports  Mexico   
10                    326--Plastics & Rubber Products  Imports  Mexico   
11                               910--Waste And Scrap  Imports  Mexico   
12                               311--

In [6]:
import pandas as pd

# Assuming 'industry_trade_df' is your dataframe
def melt_data_for_all_states(df):
    # Melt the data for both Imports and Exports
    df_melted = df.melt(id_vars=['Product', 'Flow', 'State'], 
                        value_vars=[str(year) for year in range(2009, 2024)], 
                        var_name='year', 
                        value_name='trade_value')
    
    # Convert the 'year' column to integer for easier manipulation
    df_melted['year'] = df_melted['year'].astype(int)
    
    # Create a dummy variable for the post-USMCA period (1 if year >= 2020, else 0)
    df_melted['post'] = df_melted['year'].apply(lambda x: 1 if x >= 2020 else 0)

    # Create a time variable relative to the intervention (e.g., time = year - 2020)
    df_melted['time'] = df_melted['year'] - 2020

    return df_melted

# Melt the data for all states
industry_trade_melted_df = melt_data_for_all_states(industry_trade_df)

# Check the melted data
print(industry_trade_melted_df.head())


                           Product     Flow    State  year  trade_value  post  \
0               0--All Merchandise  Imports  Alabama  2009        671.0     0   
1    336--Transportation Equipment  Imports  Alabama  2009        280.0     0   
2                   211--Oil & Gas  Imports  Alabama  2009          0.0     0   
3  331--Primary Metal Manufactures  Imports  Alabama  2009          6.0     0   
4   332--Fabricated Metal Products  Imports  Alabama  2009         20.0     0   

   time  
0   -11  
1   -11  
2   -11  
3   -11  
4   -11  


In [7]:
import pandas as pd
import statsmodels.formula.api as smf

# Filter data for Arizona
arizona_data = industry_trade_melted_df[industry_trade_melted_df['State'] == 'Arizona']

# Initialize a list to store the results
its_results = []

# Specify whether to analyze 'Imports' or 'Exports' (change this variable as needed)
flow_type = 'Imports'  # Change this to 'Exports' if you want to analyze exports

# Filter the data for the specified flow type (Imports or Exports)
arizona_data_flow = arizona_data[arizona_data['Flow'] == flow_type]

# Loop over each unique product for the specified flow type
for (product, flow), group in arizona_data_flow.groupby(['Product', 'Flow']):
    # Create a dummy variable for the post-USMCA period (1 if year >= 2020, else 0)
    group['post'] = group['year'].apply(lambda x: 1 if x >= 2020 else 0)

    # Create a time variable relative to the intervention (e.g., time = year - 2020)
    group['time'] = group['year'] - 2020
    
    # Drop rows with missing trade_value (for this product and flow)
    group = group.dropna(subset=['trade_value'])

    # Require a minimum number of observations for a meaningful regression
    if group.shape[0] < 10:
        continue

    # Run the ITS regression: trade_value ~ post + time
    model = smf.ols(formula="trade_value ~ post + time", data=group).fit()
    
    # Append results for each product and flow
    its_results.append({
        'product': product,
        'flow': flow,
        'coef_post': model.params.get('post', None),
        'p_value_post': model.pvalues.get('post', None),
    })

# Convert results into a DataFrame
its_results_df = pd.DataFrame(its_results)

# Show products with a statistically significant change in trade value post-USMCA (p-value < 0.05)
significant_results = its_results_df[its_results_df['p_value_post'] < 0.05]

print(f"Statistically significant {flow_type} products from Arizona post-USMCA:")
print(significant_results)


Statistically significant Imports products from Arizona post-USMCA:
                                product     flow   coef_post  p_value_post
6                  212--Minerals & Ores  Imports  -95.673913      0.006519
7                  311--Processed Foods  Imports   85.814229      0.004349
8     312--Beverages & Tobacco Products  Imports  -56.027668      0.017186
9               313--Textiles & Fabrics  Imports   10.207510      0.004179
11           315--Apparel & Accessories  Imports  -38.189723      0.031467
13                   321--Wood Products  Imports   13.424901      0.022432
14                           322--Paper  Imports   33.492095      0.020766
23  334--Computer & Electronic Products  Imports -238.211462      0.025505
26            337--Furniture & Fixtures  Imports  -65.227273      0.002271
28                 910--Waste And Scrap  Imports    9.460474      0.016817


In [8]:
import pandas as pd
import statsmodels.formula.api as smf

# Filter data for Arizona
rhode_island_data = industry_trade_melted_df[industry_trade_melted_df['State'] == 'Rhode Island']

# Initialize a list to store the results
its_results = []

# Specify whether to analyze 'Imports' or 'Exports' (change this variable as needed)
flow_type = 'Imports'  # Change this to 'Exports' if you want to analyze exports

# Filter the data for the specified flow type (Imports or Exports)
rhode_island_data_flow = rhode_island_data[rhode_island_data['Flow'] == flow_type]

# Loop over each unique product for the specified flow type
for (product, flow), group in rhode_island_data_flow.groupby(['Product', 'Flow']):
    # Create a dummy variable for the post-USMCA period (1 if year >= 2020, else 0)
    group['post'] = group['year'].apply(lambda x: 1 if x >= 2020 else 0)

    # Create a time variable relative to the intervention (e.g., time = year - 2020)
    group['time'] = group['year'] - 2020
    
    # Drop rows with missing trade_value (for this product and flow)
    group = group.dropna(subset=['trade_value'])

    # Require a minimum number of observations for a meaningful regression
    if group.shape[0] < 0:
        continue

    # Run the ITS regression: trade_value ~ post + time
    model = smf.ols(formula="trade_value ~ post + time", data=group).fit()
    
    # Append results for each product and flow
    its_results.append({
        'product': product,
        'flow': flow,
        'coef_post': model.params.get('post', None),
        'p_value_post': model.pvalues.get('post', None),
    })

# Convert results into a DataFrame
its_results_df = pd.DataFrame(its_results)

# Show products with a statistically significant change in trade value post-USMCA (p-value < 0.05)
significant_results = its_results_df[its_results_df['p_value_post'] < 0.05]

print(f"Statistically significant {flow_type} products from Rhode Island post-USMCA:")
print(significant_results)

Statistically significant Imports products from Rhode Island post-USMCA:
                                product     flow    coef_post  p_value_post
0                    0--All Merchandise  Imports -1382.335968      0.001568
1            111--Agricultural Products  Imports     0.652174      0.013291
4     114--Fish & Other Marine Products  Imports    -5.891304      0.043612
20       332--Fabricated Metal Products  Imports    -8.023715      0.000039
21    333--Machinery, Except Electrical  Imports   -31.756917      0.000701
22  334--Computer & Electronic Products  Imports    14.413043      0.036838
24        336--Transportation Equipment  Imports -1420.814229      0.001282


In [9]:
import pandas as pd
import statsmodels.formula.api as smf

# Filter data for Nevada
hawaii_data = industry_trade_melted_df[industry_trade_melted_df['State'] == 'Hawaii']

# Initialize a list to store the results
its_results = []

# Specify whether to analyze 'Imports' or 'Exports' (change this variable as needed)
flow_type = 'Exports'  # Change this to 'Imports' if you want to analyze imports

# Filter the data for the specified flow type (Imports or Exports)
hawaii_data_flow = hawaii_data[hawaii_data['Flow'] == flow_type]

# Verify the filter
print(hawaii_data_flow['Flow'].unique())  # Should print only ['Exports']

# Set a trade volume threshold (this value will depend on your data)
trade_value_threshold = 1  # Change this value as needed

# Loop over each unique product for the specified flow type
for (product, flow), group in hawaii_data_flow.groupby(['Product', 'Flow']):  # Fix variable name
    # Aggregate total trade value for each product (over all years)
    total_trade_value = group['trade_value'].sum()
    
    # Only consider products with trade values greater than the threshold
    if total_trade_value < trade_value_threshold:
        continue

    # Create a dummy variable for the post-USMCA period (1 if year >= 2020, else 0)
    group['post'] = group['year'].apply(lambda x: 1 if x >= 2020 else 0)

    # Create a time variable relative to the intervention (e.g., time = year - 2020)
    group['time'] = group['year'] - 2020
    
    # Drop rows with missing trade_value (for this product and flow)
    group = group.dropna(subset=['trade_value'])

    # Require a minimum number of observations for a meaningful regression
    if group.shape[0] < 10:
        continue

    # Run the ITS regression: trade_value ~ post + time
    model = smf.ols(formula="trade_value ~ post + time", data=group).fit()
    
    # Append results for each product and flow
    its_results.append({
        'product': product,
        'flow': flow,
        'coef_post': model.params.get('post', None),
        'p_value_post': model.pvalues.get('post', None),
        'total_trade_value': total_trade_value  # Include total trade value for reference
    })

# Convert results into a DataFrame
its_results_df = pd.DataFrame(its_results)

# Show products with a statistically significant change in trade value post-USMCA (p-value < 0.05)
significant_results = its_results_df[its_results_df['p_value_post'] < 0.05]

# Sort the significant results by total trade value
significant_results = significant_results.sort_values(by='total_trade_value', ascending=False)

print(f"Statistically significant {flow_type} products from Hawaii post-USMCA (with high trade values):")
print(significant_results)


['Exports']
Statistically significant Exports products from Hawaii post-USMCA (with high trade values):
                                             product     flow  coef_post  \
0                                 0--All Merchandise  Exports   1.691700   
3  335--Electrical Equipment, Appliances & Compon...  Exports   1.235178   

   p_value_post  total_trade_value  
0      0.012872               32.0  
3      0.000108                5.0  


In [10]:
import pandas as pd
import statsmodels.formula.api as smf

# Filter data for Arizona
massachusetts_data = industry_trade_melted_df[industry_trade_melted_df['State'] == 'Massachusetts']

# Initialize a list to store the results
its_results = []

# Loop over each unique product
for product, group in massachusetts_data.groupby('Product'):
    # Separate imports and exports data
    exports_data = group[group['Flow'] == 'Exports']
    imports_data = group[group['Flow'] == 'Imports']

    # Merge the imports and exports data on year
    merged_data = pd.merge(exports_data, imports_data, on='year', suffixes=('_exports', '_imports'))

    # Calculate the trade balance (exports - imports)
    merged_data['trade_balance'] = merged_data['trade_value_exports'] - merged_data['trade_value_imports']

    # Set a trade volume threshold (this value will depend on your data)
    trade_value_threshold = 300  # Change this value as needed

    # Aggregate total trade value for each product (over all years)
    total_trade_value = merged_data['trade_value_exports'].sum() + merged_data['trade_value_imports'].sum()

    # Only consider products with trade values greater than the threshold
    if total_trade_value < trade_value_threshold:
        continue

    # Create a dummy variable for the post-USMCA period (1 if year >= 2020, else 0)
    merged_data['post'] = merged_data['year'].apply(lambda x: 1 if x >= 2020 else 0)

    # Create a time variable relative to the intervention (e.g., time = year - 2020)
    merged_data['time'] = merged_data['year'] - 2020
    
    # Drop rows with missing trade values (for this product)
    merged_data = merged_data.dropna(subset=['trade_balance'])

    # Require a minimum number of observations for a meaningful regression
    if merged_data.shape[0] < 10:
        continue

    # Run the ITS regression: trade_balance ~ post + time
    model = smf.ols(formula="trade_balance ~ post + time", data=merged_data).fit()
    
    # Append results for each product
    its_results.append({
        'product': product,
        'coef_post': model.params.get('post', None),
        'p_value_post': model.pvalues.get('post', None),
        'coef_time': model.params.get('time', None),
        'p_value_time': model.pvalues.get('time', None),
        'total_trade_value': total_trade_value  # Include total trade value for reference
    })

# Convert results into a DataFrame
its_results_df = pd.DataFrame(its_results)

# Show products with a statistically significant change in trade balance post-USMCA (p-value < 0.05)
significant_results = its_results_df[its_results_df['p_value_post'] < 0.05]

# Sort the significant results by total trade value
significant_results = significant_results.sort_values(by='total_trade_value', ascending=False)

print("Products from Massachusetts with statistically significant change in trade balance post-USMCA:")
print(significant_results)


Products from Massachusetts with statistically significant change in trade balance post-USMCA:
                            product   coef_post  p_value_post  coef_time  \
0                0--All Merchandise -722.037549      0.009815  54.856522   
9   331--Primary Metal Manufactures -276.790514      0.027109  72.247826   
10   332--Fabricated Metal Products  -58.059289      0.033138   1.926087   
3           313--Textiles & Fabrics  -42.648221      0.028518   5.965217   
14    336--Transportation Equipment  -44.092885      0.009431   3.860870   

    p_value_time  total_trade_value  
0       0.042075            85321.0  
9       0.000033             5025.0  
10      0.450552             2091.0  
3       0.005265             1309.0  
14      0.021571              994.0  


In [11]:
import pandas as pd
import statsmodels.formula.api as smf

# Initialize a list to store the results
its_results = []

# Filter data for the product "0--All Merchandise"
all_merchandise_data = industry_trade_melted_df[industry_trade_melted_df['Product'] == '0--All Merchandise']

# Loop over each state
for state, state_data in all_merchandise_data.groupby('State'):
    # Filter for imports only
    imports_data = state_data[state_data['Flow'] == 'Imports']

    # Set a trade volume threshold (adjust as needed)
    trade_value_threshold = 2000  # Change this value as needed

    # Aggregate total trade value for the state (over all years)
    total_trade_value = imports_data['trade_value'].sum()

    # Only consider states with total trade values greater than the threshold
    if total_trade_value < trade_value_threshold:
        continue

    # Create a dummy variable for the post-USMCA period (1 if year >= 2020, else 0)
    imports_data['post'] = imports_data['year'].apply(lambda x: 1 if x >= 2020 else 0)

    # Create a time variable relative to the intervention (e.g., time = year - 2020)
    imports_data['time'] = imports_data['year'] - 2020
    
    # Drop rows with missing trade values
    imports_data = imports_data.dropna(subset=['trade_value'])

    # Require a minimum number of observations for a meaningful regression
    if imports_data.shape[0] < 10:
        continue

    # Run the ITS regression: trade_value ~ post + time
    model = smf.ols(formula="trade_value ~ post + time", data=imports_data).fit()
    
    # Append results for each state
    its_results.append({
        'state': state,
        'coef_post': model.params.get('post', None),
        'p_value_post': model.pvalues.get('post', None),
        'total_trade_value': total_trade_value  # Include total trade value for reference
    })

# Convert results into a DataFrame
its_results_df = pd.DataFrame(its_results)

# Show states with a statistically significant change in imports for "0--All Merchandise" post-USMCA (p-value < 0.05)
significant_results = its_results_df[its_results_df['p_value_post'] < 0.05]

# Sort the significant results by total trade value
significant_results = significant_results.sort_values(by='total_trade_value', ascending=False)

print("States with a statistically significant change in '0--All Merchandise' imports post-USMCA:")
print(significant_results)


C:\Users\isaac\AppData\Local\Temp\ipykernel_11772\756536687.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imports_data['post'] = imports_data['year'].apply(lambda x: 1 if x >= 2020 else 0)
C:\Users\isaac\AppData\Local\Temp\ipykernel_11772\756536687.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imports_data['time'] = imports_data['year'] - 2020
C:\Users\isaac\AppData\Local\Temp\ipykernel_11772\756536687.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

States with a statistically significant change in '0--All Merchandise' imports post-USMCA:
           state     coef_post  p_value_post  total_trade_value
18      Michigan -13465.142292      0.000935           686549.0
8        Georgia   2801.887352      0.003575           109242.0
14      Kentucky  -1131.211462      0.036539            82070.0
27      New York   -687.035573      0.011406            49227.0
20   Mississippi   2438.166008      0.014068            44495.0
16      Maryland   1000.118577      0.026484            39289.0
34  Rhode Island  -1382.335968      0.001568            21678.0
39      Virginia    924.306324      0.032965            19447.0
4       Colorado   -724.426877      0.006702            19367.0
26    New Mexico    961.790514      0.003308            15177.0
23        Nevada    286.490119      0.027849            10016.0
6       Delaware    561.632411      0.026127             7030.0


C:\Users\isaac\AppData\Local\Temp\ipykernel_11772\756536687.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imports_data['post'] = imports_data['year'].apply(lambda x: 1 if x >= 2020 else 0)
C:\Users\isaac\AppData\Local\Temp\ipykernel_11772\756536687.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imports_data['time'] = imports_data['year'] - 2020
C:\Users\isaac\AppData\Local\Temp\ipykernel_11772\756536687.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [12]:
file_path = r'C:\Users\isaac\Downloads\Trade_Data\total_trade_with_mexico.csv'

total_trade_df = pd.read_csv(file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\isaac\\Downloads\\Trade_Data\\total_trade_with_mexico.csv'

In [ ]:
import pandas as pd

# Summarize industry_trade_df to get total annual imports and exports for each state
industry_summary = industry_trade_df[industry_trade_df['Product'] == '0--All Merchandise']
industry_summary = industry_summary.groupby(['State', 'Flow']).sum().reset_index()

# Pivot the data to have separate columns for Imports and Exports
industry_pivot = industry_summary.pivot(index='State', columns='Flow', values=industry_summary.columns[2:])
industry_pivot.columns = [f'{flow}_{year}' for flow, year in industry_pivot.columns]
industry_pivot.reset_index(inplace=True)

# Merge with total_trade_df on State
merged_df = total_trade_df.merge(industry_pivot, on='State', how='left')

# Identify discrepancies in Imports and Exports for each year
discrepancies = []
for year in range(2009, 2025):
    import_col = f'Imports_{year}'
    export_col = f'Exports_{year}'
    if import_col in merged_df.columns and export_col in merged_df.columns:
        merged_df[f'Import_Discrepancy_{year}'] = merged_df[import_col] - merged_df['Imports']
        merged_df[f'Export_Discrepancy_{year}'] = merged_df[export_col] - merged_df['Exports']
        
        # Store discrepancies where values do not match
        discrepancies.append(merged_df[['State', f'Import_Discrepancy_{year}', f'Export_Discrepancy_{year}']]
                           .query(f'Import_Discrepancy_{year} != 0 or Export_Discrepancy_{year} != 0'))

# Concatenate and display discrepancies
discrepancies_df = pd.concat(discrepancies) if discrepancies else pd.DataFrame()
print("Discrepancies in Import/Export data:")
print(discrepancies_df)


In [ ]:
industry_trade_df.columns = industry_trade_df.columns.str.strip()
total_trade_df.columns = total_trade_df.columns.str.strip()

print(industry_trade_df.columns)

In [ ]:
# Convert Imports and Exports to numeric, forcing errors to NaN if necessary
total_trade_df["Imports"] = pd.to_numeric(total_trade_df["Imports"], errors="coerce")
total_trade_df["Exports"] = pd.to_numeric(total_trade_df["Exports"], errors="coerce")

# total_trade_df["Imports"] = total_trade_df["Imports"].str.replace(",", "").astype(float)
# total_trade_df["Exports"] = total_trade_df["Exports"].str.replace(",", "").astype(float)

In [ ]:
print(total_trade_df.dtypes)

# State       object
# Year         int64
# Imports    float64
# Exports    float64
# dtype: object

In [ ]:
industry_trade_melted = pd.melt(
    industry_trade_df, 
    id_vars=["State", "Product", "Flow"], 
    value_vars=industry_trade_df.columns[3:],  # Assuming the data starts from year columns
    var_name="Year", 
    value_name="trade_value"
)


In [ ]:
# Convert the 'Year' column to integer in both DataFrames
total_trade_df["Year"] = total_trade_df["Year"].astype(int)
industry_trade_melted["Year"] = industry_trade_melted["Year"].astype(int)

# Now perform the merge
merged_comparison = total_trade_df.merge(
    industry_trade_melted[industry_trade_melted["Product"] == "0--All Merchandise"], 
    how="left",  # or 'right' to see if data is missing on one side
    on=["State", "Year"]
)

# Replace NaN values in Imports, Exports, and trade_value with 0
merged_comparison["Imports"] = merged_comparison["Imports"].fillna(0)
merged_comparison["Exports"] = merged_comparison["Exports"].fillna(0)
merged_comparison["trade_value"] = merged_comparison["trade_value"].fillna(0)

# Now, calculate the differences
merged_comparison["Import_Diff"] = merged_comparison["Imports"] - merged_comparison["trade_value"]
merged_comparison["Export_Diff"] = merged_comparison["Exports"] - merged_comparison["trade_value"]

# Show only discrepancies
discrepancies = merged_comparison[(merged_comparison["Import_Diff"] != 0) | (merged_comparison["Export_Diff"] != 0)]
print(discrepancies)


In [ ]:
# Check the merged data for any null or mismatched values in key columns
print(merged_comparison[merged_comparison["Imports"] == 0][["State", "Year", "Imports", "Exports", "trade_value"]])
print(merged_comparison[merged_comparison["Exports"] == 0][["State", "Year", "Imports", "Exports", "trade_value"]])

# Investigate a specific state and year for discrepancies
state_year_data = merged_comparison[(merged_comparison["State"] == "California") & 
                                    (merged_comparison["Year"] == 2024)]
print(state_year_data)


In [ ]:
# import statsmodels.formula.api as smf

# def perform_its_analysis(df_melted):
#     its_results = []

#     # Get unique states
#     states = df_melted['State'].unique()

#     # Loop over each state
#     for state in states:
#         state_data = df_melted[df_melted['State'] == state].copy()
        
#         # Loop over each product and flow (import or export)
#         for flow in ['Imports', 'Exports']:
#             flow_data = state_data[state_data['Flow'] == flow].copy()
            
#             # Drop rows with missing trade_value
#             flow_data = flow_data.dropna(subset=['trade_value'])

#             # Require a minimum number of observations for a meaningful regression
#             if flow_data.shape[0] < 10:
#                 continue
            
#             # Loop over each product
#             products = flow_data['Product'].unique()
#             for product in products:
#                 product_data = flow_data[flow_data['Product'] == product].copy()
                
#                 # Run the ITS regression: trade_value ~ post + time
#                 model = smf.ols(formula="trade_value ~ post + time", data=product_data).fit()
                
#                 # Store the results for each product
#                 its_results.append({
#                     'state': state,
#                     'product': product,
#                     'flow': flow,
#                     'coef_post': model.params.get('post', None),
#                     'p_value_post': model.pvalues.get('post', None)
#                 })

#     # Convert the results into a DataFrame
#     its_results_df = pd.DataFrame(its_results)

#     # Print products with a statistically significant change (p-value < 0.05) post-USMCA
#     print("Products with significant level changes post-USMCA (p-value < 0.05) using ITS regression:")
#     print(its_results_df[its_results_df['p_value_post'] < 0.05])

# # Perform the analysis
# perform_its_analysis(industry_trade_melted_df)
